In [1]:
import pandas as pd

data = pd.read_csv("complaints_fixed.csv")
data

,text,category,region,organization
0,ขอหน่วยงานที่เกี่ยวข้องวางท่อส่งน้ำสำหรับขยายเ...,น้ำ,สระบุรี,สปน.
1,ขอให้หน่วยงานที่เกี่ยวข้องดำเนินการแก้ไขปัญหาร...,อื่น ๆ,ทุกภูมิภาค,กต.
2,ขอให้ช่วยเหลือเยียวยาประชาชนที่ได้รับผลกระทบจา...,สาธารณภัย,สุพรรณบุรี,กษ.
3,ขอให้รัฐบาลทบทวนการแก้ไขปัญหาประชาคมกระเหรี่ยง...,ที่ดินทำกิน,เพชรบุรี,ทส.
4,ขอให้พิจารณาดำเนินการแก้ไขปัญหาการลักลอบเล่นพน...,อาชญากรรม ยาเสพติดและการพนัน,กรุงเทพมหานคร,ตร.
...,...,...,...,...
1952,ขอให้พิจารณากำกับดูแลและเร่งดำเนินการจัดทำร่าง...,การเมือง,ทุกภูมิภาค,สลค.
1953,ขอให้หน่วยงานที่เกี่ยวข้องบูรณาการการทำงานร่วม...,สวัสดิการ,ทุกภูมิภาค,กต.
1954,ขอให้หน่วยงานที่เกี่ยวข้องดำเนินการฟื้นฟูเศรษฐ...,เศรษฐกิจ การคลัง,ภาคตะวันออก,คค.
1955,ขอความอนุเคราะห์ให้จัดระบบประปาหมู่บ้าน ม.6 บ้...,น้ำ,เชียงใหม่,มท.


In [2]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(data, test_size=0.3, stratify=data["category"], random_state=0)
train_data.shape[0], test_data.shape[0]

(1369, 588)

In [3]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from newmm_tokenizer.tokenizer import word_tokenize

def create_pipeline():
    return make_pipeline(
        # CountVectorizer(),
        # TfidfVectorizer(),
        # TfidfVectorizer(ngram_range=(1, 3), min_df=5, max_df=0.5),
        CountVectorizer(tokenizer=word_tokenize),
        # TfidfVectorizer(tokenizer=word_tokenize, ngram_range=(1, 3), min_df=5, max_df=0.5),

        # SelectKBest(k=1000),
        
        MultinomialNB()
        # AdaBoostClassifier(estimator=LogisticRegression(), n_estimators=10, random_state=0)
        # RandomForestClassifier(n_estimators=10, random_state=0)
    )

pipeline = create_pipeline()
pipeline.fit(train_data["text"], train_data["category"])
pipeline.score(train_data["text"], train_data["category"])

d:\6510451000\machine-learning-mini-project\.venv\Lib\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.7355734112490869

In [4]:
pipeline.steps[-1][1].n_features_in_

5730

In [5]:
pipeline.score(test_data["text"], test_data["category"])

0.5153061224489796

In [6]:
pipeline.predict(["น้ำท่วม"])[0]

'สาธารณภัย'

In [7]:
pipeline.predict(["ถนนทรุด"])[0]

'โครงสร้างพื้นฐานและสาธารณูปโภค'

In [8]:
predicted = pipeline.predict_proba(["ถนนทรุด"])[0]
predicted

array([1.37341547e-03, 1.62997949e-04, 8.20843096e-02, 7.30108134e-04,
       1.99776217e-03, 2.72194596e-02, 2.47787543e-03, 2.38529696e-03,
       2.39528903e-03, 6.33314697e-03, 3.24298542e-03, 8.73826430e-04,
       1.13739562e-02, 2.06691377e-02, 2.04958899e-03, 6.03198803e-03,
       1.10578342e-03, 4.49198052e-03, 4.64284290e-04, 2.25705411e-02,
       3.96800700e-03, 1.98016765e-01, 4.40140762e-03, 3.02502941e-03,
       6.51061203e-03, 9.80854037e-03, 3.82935892e-02, 1.32516556e-03,
       1.24960457e-03, 8.61928911e-03, 2.00292804e-03, 5.19330535e-01,
       1.14580527e-03, 2.26898828e-03])

In [9]:
from collections import OrderedDict

OrderedDict(sorted(zip(pipeline.classes_, predicted), key=lambda x: x[1])[::-1])

OrderedDict([('โครงสร้างพื้นฐานและสาธารณูปโภค', 0.5193305354915263),
             ('สาธารณภัย', 0.1980167646618376),
             ('การคมนาคม การจราจรหรืออุบัติเหตุ', 0.08208430961517234),
             ('อื่น ๆ', 0.03829358920736119),
             ('การบริหารราชการแผ่นดิน', 0.027219459595867055),
             ('ศิลปะ วัฒนธรรม ศาสนา', 0.022570541063418305),
             ('น้ำ', 0.02066913766988561),
             ('ที่ดินทำกิน', 0.011373956239822219),
             ('อาชญากรรม ยาเสพติดและการพนัน', 0.0098085403656902),
             ('เศรษฐกิจ การคลัง', 0.008619289114140187),
             ('หนี้สิน เศรษฐกิจฐานราก', 0.006510612027674902),
             ('การเมือง', 0.00633314697206917),
             ('ผู้มีอิทธิพล การทุจริตและประพฤติมิชอบ', 0.006031988027966285),
             ('ราคาสินค้าเกษตร', 0.00449198052183372),
             ('สาธารณสุข', 0.004401407616830301),
             ('สวัสดิการ', 0.003968006995724535),
             ('ความมั่นคง', 0.0032429854179133488),
             ('สิ่งแวดล้อม